# 线性回归的简洁实现

基于 PyTorch 开源框架，可以自动化基于梯度的学习算法中重复性的工作。

## 生成数据集
首先，生成数据集

In [7]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [12]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 读取数据集

调用框架中现有的函数来读取数据，将`features`和`labels`作为参数，并通过数据迭代器指定`batch_size`。
此外，布尔值`is_train`表示是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [13]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [14]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

为了验证是否正常工作，读取并打印第一个小批量样本。
这里使用`iter`构造Python迭代器，并使用`next`从迭代器中获取第一项。

In [15]:
next(iter(data_iter))

[tensor([[-0.9457,  0.0674],
         [-1.6669, -1.0045],
         [ 0.4060, -0.0388],
         [-0.9977,  0.2865],
         [ 0.0603,  0.8243],
         [-0.2106, -1.3721],
         [-0.1296,  2.8483],
         [ 1.2148, -1.0436],
         [ 0.3526, -1.7392],
         [ 0.7038,  0.7988]]),
 tensor([[ 2.0755],
         [ 4.2878],
         [ 5.1374],
         [ 1.2474],
         [ 1.5300],
         [ 8.4427],
         [-5.7454],
         [10.1777],
         [10.8107],
         [ 2.8931]])]

## 定义模型

对于标准深度学习模型，可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。
我们首先定义一个模型变量`net`，它是一个`Sequential`类的实例。
`Sequential`类将多个层串联在一起，当给定输入数据时，`Sequential`实例将数据传入到第一层，然后将第一层的输出作为第二层的输入，以此类推。
在下面的例子中，模型只包含一个层，因此实际上不需要`Sequential`，但是由于以后几乎所有的模型都是多层的，在这里使用`Sequential`会让你熟悉“标准的流水线”。

在 PyTorch 中，全连接层在`Linear`类中定义。
值得注意的是，我们将两个参数传递到`nn.Linear`中，第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为 1。

In [16]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

## 参数初始化

在使用`net`之前，需要初始化模型参数，也就是线性回归模型中的权重和偏置。
深度学习框架通常有预定义的方法来初始化参数，这里指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为零。

正如我构造`nn.Linear`时指定输入和输出尺寸一样，能直接访问参数以设定它们的初始值。
通过`net[0]`选择网络中的第一个图层，然后使用`weight.data`和`bias.data`方法访问参数。
还可以使用替换方法`normal_`和`fill_`来重写参数值。

In [17]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数，默认情况下，它返回所有样本损失的平均值。

In [18]:
loss = nn.MSELoss()

## 定义优化器
小批量随机梯度下降算法是一种优化神经网络的标准工具，PyTorch在`optim`模块中实现了该算法的许多变种。
当需要实例化一个`SGD`时，要指定优化的参数（可通过`net.parameters()`从模型中获得）以及优化算法所需的超参数字典。
小批量随机梯度下降只需要设置`lr`值，这里设置为0.03。

In [19]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

通过深度学习框架来实现的模型只需要相对较少的代码，并且不必单独分配参数、不必定义损失函数，也不必手动实现小批量随机梯度下降。
当需要更复杂的模型时，高级函数的优势将大大增加。
当有了所有的基本组件，训练过程代码与从零开始实现时所做的非常相似。

在每个迭代周期里，将完整遍历一次数据集（`train_data`），不停地从中获取一个batchsize的输入和相应的标签。
对于每一个小批量，会进行以下步骤:

- 通过调用`net(X)`生成预测并计算损失`l`（前向传播）。
- 通过进行反向传播来计算梯度。
- 通过调用优化器来更新模型参数。

为了更好的衡量训练效果，计算每个迭代周期后的损失，并打印它来监控训练过程。

In [20]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000221
epoch 2, loss 0.000104
epoch 3, loss 0.000105


下面，我们比较生成数据集的真实参数和通过有限数据训练获得的模型参数。
要访问参数，首先从`net`访问所需的层，然后读取该层的权重和偏置。
正如在从零开始实现中一样，我们估计得到的参数与生成数据的真实参数非常接近。

In [21]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0006, 0.0004])
b的估计误差： tensor([3.0041e-05])
